# Artificial Neural Networks

ip-biocode $\cdot$ March 26, 2022

In this exercise, I build a neural network from scratch and code how it performs predictions using forward propagation. This is only to help with understanding the underlying mechanisms. All deep learning libraries have the entire training and prediction processes implemented, and so in practice you wouldn't really need to build a neural network from scratch.

## Forward Propagation: Simple Example

<img src="http://cocl.us/neural_network_example" alt="Neural Network Example" width=600px>

Here is a simple neural network that takes two inputs, has one hidden layer with two nodes, and an output layer with one node.

### Initialize weights and biases
Begin by randomly initializing weights and biases in the network. We have 6 weights and 3 biases.

In [2]:
import numpy as np

weights = np.around(np.random.uniform(size=6), decimals=2)
biases = np.around(np.random.uniform(size=3), decimals=2)

print(weights)
print(biases)

[0.95 0.02 0.92 0.23 0.5  0.12]
[0.43 0.25 0.47]


### Compute weighted sums
Now that we have the weights and the biases defined for the network, let's compute the output for a given input, $x\_1$ and $x\_2$.


In [3]:
x_1 = 0.5 # input 1
x_2 = 0.85 # input 2

print('x1 is {} and x2 is {}'.format(x_1, x_2))

x1 is 0.5 and x2 is 0.85


Compute the wighted sum of the inputs at the first node of the hidden layer, $z_{1, 1}$. The weights are $w_1$ and $w_2$.


In [8]:
z_11 = x_1 * weights[0] + x_2 * weights[1] + biases[0]

print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_11))

The weighted sum of the inputs at the first node in the hidden layer is 0.9219999999999999


Next compute the weighted sum of the inputs at the second node of the hidden layer, $z_{1, 2}$. The weights are $w_3$ and $w_4$.


In [10]:
z_12 = x_1 * weights[2] + x_2 * weights[3] + biases[1]

print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_12))

The weighted sum of the inputs at the first node in the hidden layer is 0.9055


### Compute node activation
Next, assuming a sigmoid activation function, compute the activation of the first node, $a_{1, 1}$, in the hidden layer.


In [11]:
a_11 = 1.0 / (1.0 + np.exp(-z_11))

print('The activation of the first node in the hidden layer is {}'.format(np.around(a_11, decimals=4)))

The activation of the first node in the hidden layer is 0.7154


Compute the activation of the second node, $a_{1, 2}$, in the hidden layer.


In [12]:
a_12 = 1.0 / (1.0 + np.exp(-z_12))

print('The activation of the first node in the hidden layer is {}'.format(np.around(a_12, decimals=4)))

The activation of the first node in the hidden layer is 0.7121


Now these activations will serve as the inputs to the output layer. Compute the weighted sum of these inputs to the node in the output layer.

In [13]:
z_2 = z_11*weights[4] + z_12*weights[5] + biases[2]

print('The weighted sum of the inputs at the node in the output layer is {}'.format(np.around(z_2, decimals=4)))

The weighted sum of the inputs at the node in the output layer is 1.0397


Finally, compute the output of the network as the activation of the node in the output layer.


In [14]:
a_2 = 1.0 / (1.0 + np.exp(-z_2))

print('The output of the network for x1=0.5 and x2=0.85 is {}'.format(np.around(a_2, decimals=4)))

The output of the network for x1=0.5 and x2=0.85 is 0.7388


## A General Network
Obviously, neural networks for real problems are composed of many hidden layers and many more nodes in each layer. So, we can't continue making predictions using this very inefficient approach of computing the weighted sum at each node and the activation of each node manually.

In order to make predictions automatically, we should generalize our network. A general network would take $n$ inputs, would have many hidden layers, each hidden layer having $m$ nodes, and would have an output layer. Although the network is showing one hidden layer, we will code the network to have many hidden layers. Similarly, although the network shows an output layer with one node, we will code the network to have more than one node in the output layer.

<img src="http://cocl.us/general_neural_network" alt="Neural Network General" width=600px>


